# xml 파일 쪼개기

### 각 application이 하나의 xml 파일 그 자체    
__=> 하나씩 잘라야 함__

In [1]:
import os
import re
import xml.etree.ElementTree as et

os.chdir('J:/data')

In [2]:
f=open('sample.xml', 'r')
endtag_regex = re.compile('^<!DOCTYPE (.*) SYSTEM')
endtag = ''
doc=''

In [10]:
line = f.readline()
doc += line

In [11]:
endtag

[]

In [12]:
endtag_regex.findall(line)

['us-patent-application']

In [13]:
endtag = endtag_regex.findall(line) if not endtag else endtag
endtag

['us-patent-application']

In [14]:
not endtag

False

In [15]:
doc

'<?xml version="1.0" encoding="UTF-8"?>\n<!DOCTYPE us-patent-application SYSTEM "us-patent-application-v41-2005-08-25.dtd" [ ]>\n'

In [16]:
terminate = re.compile('^</{0}>'.format(endtag[0]))

In [17]:
terminate.findall(line)

[]

if not ~ else? 이게 뭐지?

In [ ]:
if not endtag == True:
    endtag = endtag_regex.findall(line)
else:
    endtag

endtag

아하 이렇게 쓰기 싫어서 그냥 한 줄로 끝냈구만!  
그렇다면 정리해서 함수로 만들자

In [18]:
def extract_xml_strings(filename):
    """
    Given a string [filename], opens the file and returns a generator
    that yields tuples. A tuple is of format (year, xmldoc string). A tuple
    is returned for every valid XML doc in [filename]
    """
    # search for terminating XML tag
    endtag_regex = re.compile('^<!DOCTYPE (.*) SYSTEM')
    endtag = ''
    with open(filename, 'r') as f:
        doc = ''  # initialize current XML doc to empty string
        for line in f:
            doc += line
            endtag = endtag_regex.findall(line) if not endtag else endtag   #is there any endtags?
            if not endtag:
                continue      #if there's no end tag, keep the loop going
            terminate = re.compile('^</{0}>'.format(endtag[0]))    #if there's a end tag, then check whether the doc is finished or not
            if terminate.findall(line):     #if there's a terminator
                yield (doc)                 #export doc and reinitialize a doc
                endtag = ''
                doc = ''

In [29]:
sample = extract_xml_strings('sample.xml')

In [30]:
for x in sample:
    print x

<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE us-patent-application SYSTEM "us-patent-application-v41-2005-08-25.dtd" [ ]>
<us-patent-application lang="EN" dtd-version="v4.1 2005-08-25" file="US20070050875A1-20070308.XML" status="PRODUCTION" id="us-patent-application" country="US" date-produced="20070221" date-publ="20070308">
<us-bibliographic-data-application lang="EN" country="US">
<publication-reference>
<document-id>
<country>US</country>
<doc-number>20070050875</doc-number>
<kind>A1</kind>
<date>20070308</date>
</document-id>
</publication-reference>
<application-reference appl-type="utility">
<document-id>
<country>US</country>
<doc-number>11210174</doc-number>
<date>20050822</date>
</document-id>
</application-reference>
<us-application-series-code>11</us-application-series-code>
<classifications-ipcr>
<classification-ipcr>
<ipc-version-indicator><date>20060101</date></ipc-version-indicator>
<classification-level>A</classification-level>
<section>A</section>
<class>61</class

good~ 이젠 sample.next 해서 각 doc을 parsing, ID, DN, assignee, class 추출하자

# 각 XML에서 원하는 element 뽑기

App ID, DN, Assignee, Class 필요